Calculate autocorrelation per trial (1 ACF per trial):

* calculate Pearson for all trials (all non zero spike trials)
* calculate ACF proper for all trials (also non zero)
* calculate iSTTC for all trials (also non zero) 

In [1]:
import pandas as pd
import numpy as np
import csv
from statsmodels.tsa.stattools import acf
import matplotlib.pyplot as plt
import seaborn as sns
import sys

# import from scripts
import os
os.chdir(os.path.expanduser("D:\\intr_timescales\\isttc\\scripts"))
# os.chdir(os.path.expanduser("C:\\Users\\ipoch\\Documents\\repos\\isttc\\scripts"))
from calculate_acf import acf_pearsonr, acf_sttc

### Get and prep the data

In [2]:
area = 'pfp' # pfp

In [ ]:
# binned data
results_folder = 'E:\\projects_q_30_10_2024\\isttc\\results\\monkey\\'
save_folder_binned = results_folder + 'fixation_period_1000ms\\binned\\' + area + '\\acf\\'

csv_data_file = results_folder + 'data_' + area + '_fixon_1500ms_fixation_with_empty_binned_50ms.csv'
with open(csv_data_file, newline='') as f:
    reader = csv.reader(f)
    sua_binned_list = list(reader)
    
n_binned_spike_trains = len(sua_binned_list)
print('N spike_trains in {}: {}'.format(area, n_binned_spike_trains))

# transform list to an array and to a dataframe 
sua_binned_array = np.array(sua_binned_list)
sua_binned_array = sua_binned_array[:, :-1-9] # for 1000 calc

bin_cols = ['bin_' + str(i) for i in range(sua_binned_array.shape[1]-3)]
sua_binned_df = pd.DataFrame(sua_binned_array, columns=['unit_id', 'trial_id','condition_id'] + bin_cols)
sua_binned_df = sua_binned_df.astype('int')

n_binned_units = len(sua_binned_df['unit_id'].unique())
print('n units {}'.format(n_binned_units))

sua_binned_df.head(2)

In [3]:
# non-binned data
results_folder = 'E:\\projects_q_30_10_2024\\isttc\\results\\monkey\\'
save_folder_non_binned = results_folder + 'fixation_period_1000ms\\non_binned\\' + area + '\\acf\\'

csv_data_file = results_folder + 'data_' + area + '_fixon_1500ms_with_empty_fixation.csv'
with open(csv_data_file, newline='') as f:
    reader = csv.reader(f)
    sua_non_binned_list = list(reader)
    
n_non_binned_spike_trains = len(sua_non_binned_list)
print('N spike_trains in {}: {}'.format(area, n_non_binned_spike_trains))

# transform data to a dict, key is unit_id, values is a list of spike trains (one spike train per trial)
units_dict = {}
for spike_train in sua_non_binned_list:
    spike_train_ = np.asarray(spike_train[3:]).astype(int)
    spike_train_1000 = spike_train_[spike_train_ <= 1000]
    if int(spike_train[0]) in units_dict:
        units_dict[int(spike_train[0])].append(spike_train_1000)
    else:
        units_dict[int(spike_train[0])] = []
        units_dict[int(spike_train[0])].append(spike_train_1000)

n_non_binned_units = len(units_dict)
print('n units {}'.format(n_non_binned_units))

N spike_trains in pfp: 43677
n units 543


### Calculate autocorrelation

In [4]:
n_lags = 20
acf_cols = ['acf_' + str(i) for i in range(n_lags)]
print('acf_cols {}'.format(acf_cols))

acf_cols ['acf_0', 'acf_1', 'acf_2', 'acf_3', 'acf_4', 'acf_5', 'acf_6', 'acf_7', 'acf_8', 'acf_9', 'acf_10', 'acf_11', 'acf_12', 'acf_13', 'acf_14', 'acf_15', 'acf_16', 'acf_17', 'acf_18', 'acf_19']


#### Calc ACF using Pearson 

In [ ]:
old_stdout = sys.stdout
sys.stdout = open(save_folder_binned + 'acf_trial_pearsonr_cal_log.txt', 'w')

acf_trial_pearsonr_l, unit_id_pearsonr_l, trial_id_pearsonr = [],[],[]

for idx in range(len(sua_binned_df)):
#for idx in range(10):
    print('Processing unit {}, trial {}'.format(sua_binned_df['unit_id'].values[idx], sua_binned_df['trial_id'].values[idx]))
    if np.count_nonzero(sua_binned_df[bin_cols].values[idx, :]) <= 1:
        print('WARNING: trial has {} non zero bins, nothing to correlate, skipping...'.format(np.count_nonzero(sua_binned_df[bin_cols].values[idx, :])))
    else:
        # n_lags-1 because of Pearson eq - for lag 19 only one values and P will get 0s in denominator
        acf_pearsonr_ = acf_pearsonr(sua_binned_df[bin_cols].values[idx, :], n_lags_=n_lags-1, verbose_=False) 
        acf_trial_pearsonr_l.append(acf_pearsonr_)
        unit_id_pearsonr_l.append(sua_binned_df['unit_id'].values[idx])
        trial_id_pearsonr.append(sua_binned_df['trial_id'].values[idx])

acf_pearsonr_df = pd.DataFrame(np.array(acf_trial_pearsonr_l), columns=acf_cols[:-1])
acf_pearsonr_df.insert(0, 'unit_id', unit_id_pearsonr_l)
acf_pearsonr_df.insert(1, 'trial_id', trial_id_pearsonr)

print('NaNs in acf {}'.format(acf_pearsonr_df.isnull().any().any()))
acf_pearsonr_df.head(3)

sys.stdout = old_stdout

In [ ]:
acf_pearsonr_df.to_pickle(save_folder_binned + 'acf_trial_pearsonr_df_50ms_20lags_df.pkl')

#### Calc ACF using ACF equation

In [ ]:
old_stdout = sys.stdout
sys.stdout = open(save_folder_binned + 'acf_trial_proper_cal_log.txt', 'w')

acf_trial_proper_l, unit_id_proper_l, trial_id_proper_l = [],[],[]

for idx in range(len(sua_binned_df)):
    print('Processing unit {}, trial {}'.format(sua_binned_df['unit_id'].values[idx], sua_binned_df['trial_id'].values[idx]))
    if np.count_nonzero(sua_binned_df[bin_cols].values[idx, :]) <= 1:
        print('WARNING: trial has {} non zero bins, nothing to correlate, skipping...'.format(np.count_nonzero(sua_binned_df[bin_cols].values[idx, :])))
    else:
        acf_proper = acf(sua_binned_df[bin_cols].values[idx, :], nlags=n_lags)
        acf_trial_proper_l.append(acf_proper)
        unit_id_proper_l.append(sua_binned_df['unit_id'].values[idx])
        trial_id_proper_l.append(sua_binned_df['trial_id'].values[idx])

acf_proper_df = pd.DataFrame(np.array(acf_trial_proper_l), columns=acf_cols)
acf_proper_df.insert(0, 'unit_id', unit_id_proper_l)
acf_proper_df.insert(1, 'trial_id', trial_id_proper_l)

print('NaNs in acf {}'.format(acf_proper_df.isnull().any().any()))
acf_proper_df.head(3)

sys.stdout = old_stdout

In [ ]:
acf_proper_df.to_pickle(save_folder_binned + 'acf_trial_proper_df_50ms_20lags_df.pkl')

#### Calc acf using iSTTC

In [ ]:
old_stdout = sys.stdout
sys.stdout = open(save_folder_non_binned + 'acf_trial_isttc_cal_log.txt', 'w')

lag_shift = 50
sttc_dt = 50

acf_trial_isttc_l, unit_id_isttc_l, trial_id_isttc_l = [],[],[]

for idx in range(len(sua_non_binned_list)):
    print('Processing unit {}, trial {}'.format(sua_non_binned_list[idx][0], sua_non_binned_list[idx][1]))
    spike_train_a = np.asarray(sua_non_binned_list[idx][3:]).astype(int)
    spike_train = spike_train_a[spike_train_a <= 1000]
    if len(spike_train) <= 1:
        print('WARNING: trial has {} <= 1, nothing to correlate, skipping...'.format(len(spike_train)))
    else:
        acf_isttc = acf_sttc(spike_train, n_lags, lag_shift, sttc_dt, signal_length_=lag_shift*n_lags, verbose_=False))
        acf_trial_isttc_l.append(acf_isttc)
        unit_id_isttc_l.append(sua_non_binned_list[idx][0])
        trial_id_isttc_l.append(sua_non_binned_list[idx][1])

acf_isttc_df = pd.DataFrame(np.array(acf_trial_isttc_l), columns=acf_cols)
acf_isttc_df.insert(0, 'unit_id', unit_id_isttc_l)
acf_isttc_df.insert(1, 'trial_id', trial_id_isttc_l)

print('NaNs in acf {}'.format(acf_isttc_df.isnull().any().any()))
acf_isttc_df.head(3)

sys.stdout = old_stdout

In [ ]:
acf_isttc_df

In [ ]:
acf_isttc_df.to_pickle(save_folder_non_binned + 'acf_trial_isttc_df_50ms_20lags_df.pkl')

#### Calc acf using iSTTC (concatenating trails in one signal)

In [5]:
zero_padding_len = 2000 # trial len

unit_id_l = []
spike_train_concat_l = []
signal_len_l = []

for k,v in units_dict.items():
    spike_train_concat = v[0]
    for idx, trial in enumerate(v[1:]):
        spike_train_concat = np.hstack((spike_train_concat, trial+(idx+1)*zero_padding_len))
    unit_id_l.append(k)
    spike_train_concat_l.append(spike_train_concat)
    signal_len_l.append((idx+2)*2000)

In [6]:
old_stdout = sys.stdout
sys.stdout = open(save_folder_non_binned + 'acf_trial_concat_isttc_cal_log.txt', 'w')

lag_shift = 50
sttc_dt = 50

acf_trial_concat_isttc_l, unit_id_isttc_concat_l = [],[]

for idx in range(len(spike_train_concat_l)):
    print('Processing unit {}'.format(unit_id_l[idx]))
    spike_train = spike_train_concat_l[idx]
    if len(spike_train) <= 1:
        print('WARNING: unit has {} <= 1, nothing to correlate, skipping...'.format(len(spike_train)))
    else:
        acf_isttc = acf_sttc(spike_train, n_lags, lag_shift, sttc_dt, signal_length_=signal_len_l[idx], verbose_=False)
        acf_trial_concat_isttc_l.append(acf_isttc)
        unit_id_isttc_concat_l.append(unit_id_l[idx])

acf_isttc_concat_df = pd.DataFrame(np.array(acf_trial_concat_isttc_l), columns=acf_cols)
acf_isttc_concat_df.insert(0, 'unit_id', unit_id_isttc_concat_l)

print('NaNs in acf {}'.format(acf_isttc_concat_df.isnull().any().any()))
acf_isttc_concat_df.head(3)

sys.stdout = old_stdout

In [7]:
acf_isttc_concat_df

,unit_id,acf_0,acf_1,acf_2,acf_3,acf_4,acf_5,acf_6,acf_7,acf_8,...,acf_10,acf_11,acf_12,acf_13,acf_14,acf_15,acf_16,acf_17,acf_18,acf_19
0,0,1.0,0.501834,-0.048608,-0.048620,-0.048573,-0.048387,-0.048400,-0.048412,-0.048425,...,-0.048450,-0.048462,-0.048475,-0.048487,-0.04850,-0.048512,0.0,0.0,0.0,0.0
1,1,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0
2,2,1.0,0.511305,-0.043695,0.221840,0.221812,-0.043795,-0.043828,-0.043100,-0.043133,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0
3,3,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0
4,4,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,538,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0
536,539,1.0,0.519194,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0
537,540,1.0,0.525317,-0.015792,-0.015807,-0.015823,-0.015628,-0.014849,-0.014864,-0.014879,...,-0.014909,-0.014924,-0.014939,-0.014954,-0.01497,0.000000,0.0,0.0,0.0,0.0
538,541,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0


In [8]:
acf_isttc_concat_df.to_pickle(save_folder_non_binned + 'acf_trial_isttc_concat_df_50ms_20lags_df.pkl')